In [1]:
%load_ext autoreload
%autoreload 2

import sys
from collections import OrderedDict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append("../../..")

import lib.ehr.example_datasets.mimic4 as m4

In [2]:
# create a database connection
sqluser = 'postgres'
dbname = 'mimiciv'
hostname = 'localhost'
password = 'qwerasdf'
port = '5432'

mimiciv_tables_config = m4.MIMICIVSQLTablesConfig(host=hostname, port=port, user=sqluser, password=password, dbname=dbname)
mimiciv_sql = m4.MIMICIVSQLTablesInterface(config=mimiciv_tables_config)
mimiciv_engine = mimiciv_sql.create_engine()

In [3]:
mimiciv_sql.supported_obs_variables.reset_index().to_csv("selection/obs.csv")

In [13]:
icd_map = pd.read_csv("mimic4_int_grouper_proc.csv", index_col=0)
icd_map = icd_map[icd_map.code.str.startswith("v")]
icd_map = icd_map.rename(columns={"label": "description"})
icd_map["icd_code"] = icd_map.code.str.replace("v", "").str.split(":c").str[1]
icd_map["icd_version"] = icd_map.code.str.replace("v", "").str.split(":c").str[0]
groups = list(sorted(icd_map.group.unique().tolist()))
codes = [f'group_{i}' for i in range(len(groups))]
target_desc = dict(zip(codes, groups))
icd_map["target_code"] = icd_map.group.map(dict(zip(groups, codes)))
icd_map["target_description"] = icd_map.target_code.map(target_desc)
icd_map = icd_map[["icd_code", "icd_version", "description", "target_code", "target_description"]]
icd_map = icd_map.sort_values("target_description")
icd_map.to_csv("selection/hosp_procedures.csv")
